In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nameparser import HumanName
import webbrowser
%matplotlib inline

Import CSV file of data that needs cleaning and check its shape.

In [2]:
killings = pd.read_csv('./csv_files/police_killings_original.csv')
killings.shape

(7907, 66)

# Data Cleaning

Drop rows that contain ALL null values, then drop columns that contain ALL null values

In [3]:
killings.dropna(how='all', axis=0, inplace=True)
killings.dropna(how='all', axis=1, inplace=True)
killings.shape

(7663, 27)

In [4]:
killings.columns

Index(['Victim's name', 'Victim's age', 'Victim's gender', 'Victim's race',
       'URL of image of victim', 'Date of Incident (month/day/year)',
       'Street Address of Incident', 'City', 'State', 'Zipcode', 'County',
       'Agency responsible for death', 'Cause of death',
       'A brief description of the circumstances surrounding the death',
       'Official disposition of death (justified or other)',
       'Criminal Charges?',
       'Link to news article or photo of official document',
       'Symptoms of mental illness?', 'Unarmed',
       'Alleged Weapon (Source: WaPo)', 'Alleged Threat Level (Source: WaPo)',
       'Fleeing (Source: WaPo)', 'Body Camera (Source: WaPo)',
       'WaPo ID (If included in WaPo database)', 'Off-Duty Killing?',
       'Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-content/uploads/2015/05/full-ZCTA-urban-suburban-rural-classification.xlsx )',
       'ID'],
      dtype='object')

## Cleaning column names and dropping unnecessary columns
The column names are pretty messy, let's change that

In [5]:
killings.rename(columns={
    "Victim's name":"victims_name", 
    "Victim's age":"victims_age", 
    "Victim's gender":"victims_gender", 
    "Victim's race":"victims_race",
    "URL of image of victim": "victim_img_url",
    "Date of Incident (month/day/year)":"date",
    "Street Address of Incident": "street_address",
    "City":"city",
    "State":"state",
    "Zipcode":"zipcode",
    "County":"county",
    "Agency responsible for death":"agency_resp_for_death",
    "Cause of death":"cause_of_death",
    "A brief description of the circumstances surrounding the death":"desc_of_circumstances",
    "Official disposition of death (justified or other)":"official_disposition_of_death",
    "Criminal Charges?":"criminal_charges", 
    'Link to news article or photo of official document':"news_article_link",
    'Symptoms of mental illness?':'mental_illness',
    "Unarmed":"unarmed",
    'Alleged Weapon (Source: WaPo)':'alleged_weapon',
    'Alleged Threat Level (Source: WaPo)':'threat_level',
    'Fleeing (Source: WaPo)': 'fleeing',
    'Body Camera (Source: WaPo)':'video_surveillance',
    'WaPo ID (If included in WaPo database)':'WaPo_id',
    'Off-Duty Killing?':'off_duty_killing',
    'Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-content/uploads/2015/05/full-ZCTA-urban-suburban-rural-classification.xlsx )':'geo_type',
    }, inplace=True)

Check to see what percentage of null values are contained in each column that remains.

In [6]:
(killings.isnull().sum() / len(killings)) * 100

victims_name                      0.000000
victims_age                       0.874331
victims_gender                    0.104398
victims_race                      0.000000
victim_img_url                   45.191178
date                              0.000000
street_address                    1.083127
city                              0.078298
state                             0.000000
zipcode                           0.508939
county                            0.195746
agency_resp_for_death             0.208796
cause_of_death                    0.000000
desc_of_circumstances             0.260994
official_disposition_of_death     3.340728
criminal_charges                  0.000000
news_article_link                 0.156597
mental_illness                    0.143547
unarmed                           0.000000
alleged_weapon                    0.000000
threat_level                     31.084432
fleeing                          34.138066
video_surveillance               37.439645
WaPo_id    

Because off_duty_killing is missing 97% of values, I'm going to drop it.

Because we can use our df index to ID each row, we don't need the ID column and can drop it.

I've also tried to access the WaPo database to look up cases based on WaPo_id, but it doesn't let me search using that parameter, so I'm going to drop that column too.

In [7]:
killings.drop(['WaPo_id', 'off_duty_killing', 'ID'], axis=1, inplace=True)

Let's investigate what our columns are about and perform any necessary cleaning column by column

In [8]:
killings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7663 entries, 0 to 7662
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   victims_name                   7663 non-null   object 
 1   victims_age                    7596 non-null   object 
 2   victims_gender                 7655 non-null   object 
 3   victims_race                   7663 non-null   object 
 4   victim_img_url                 4200 non-null   object 
 5   date                           7663 non-null   object 
 6   street_address                 7580 non-null   object 
 7   city                           7657 non-null   object 
 8   state                          7663 non-null   object 
 9   zipcode                        7624 non-null   float64
 10  county                         7648 non-null   object 
 11  agency_resp_for_death          7647 non-null   object 
 12  cause_of_death                 7663 non-null   o

## Cleaning column by column

### Victim's name - splitting to first and last names

In [9]:
names_without_police_in_them = killings.loc[killings['victims_name'].str.contains('police')==False, 'victims_name']

killings['first_name'] = names_without_police_in_them.apply(lambda x: HumanName(x).first)

killings['last_name'] = names_without_police_in_them.apply(lambda x: HumanName(x).last)

In [10]:
# killings[['First Name', 'Last Name']] = killings['victims_name'].loc[killings['victims_name'].str.split().str.len() == 2].str.split(expand=True)

# killings.loc[killings['victims_name'].str.split().str.len() != 2, 'First Name'] = killings['victims_name'].str.split().str[0]
# killings.loc[killings['victims_name'].str.split().str.len() != 2, 'Last Name'] = killings['victims_name'].apply(lambda x: HumanName(x).last)

# killings.loc[killings['victims_name'] == 'Name withheld by police', ['First Name', 'Last Name']] = ['Unknown', 'Unknown']

# killings[['First Name', 'Last Name']]

### Victim's age

In [11]:
killings['victims_age'].unique()

array(['28', nan, '57', '23', '37', '49', '47', '51', '27', '50', '76',
       '29', '34', '31', '52', '43', '42', '16', '35', '59', '56', '65',
       '60', '38', '24', '46', '40', '26', '61', '44', '21', '25', '41',
       '63', '22', '18', '30', '55', '66', '68', '17', '70', '58', '20',
       '54', '48', '36', '32', '33', '39', '45', '62', '19', '53', '75',
       '64', '67', '74', '77', '69', '73', '80', '14', '71', '81',
       'Unknown', '6', '78', '72', '84', '15', '89', '1', '40s', '82',
       '79', '91', '83', '13', '86', '10', '12', '95', '5', '93', '7',
       '107', '87'], dtype=object)

In [12]:
killings.loc[killings["victims_age"]=='Unknown', "victims_age"] = np.nan

In [13]:
killings.loc[killings["victims_age"]=='40s', "victims_age"] = 40.0

### Victim's gender
Let's try to determine what to do with the Null values for gender.  We'll begin by inspecting those that have a news article link present as that may have information we can use.

In [14]:
killings['victims_gender'].unique()

array(['Male', 'Female', nan, 'Transgender', 'Unknown'], dtype=object)

In [15]:
null_gender_df = killings[(killings["victims_gender"].isnull()==True) & (killings['news_article_link'].isnull()==False)]

null_gender_df

,victims_name,victims_age,victims_gender,victims_race,victim_img_url,date,street_address,city,state,zipcode,...,news_article_link,mental_illness,unarmed,alleged_weapon,threat_level,fleeing,video_surveillance,geo_type,first_name,last_name
13,Name withheld by police,NaN,NaN,Unknown race,NaN,29/12/2019,3015 Old State Rd 25,Lafayette,IN,47905.0,...,https://www.jconline.com/story/news/2019/12/30...,No,Allegedly Armed,gun,attack,not fleeing,no,Urban,NaN,NaN
112,Name withheld by police,NaN,NaN,White,NaN,29/11/2019,38300 Mountain Hwy E,Eatonville,WA,98328.0,...,https://katu.com/news/local/sheriff-suspect-ki...,No,Unclear,undetermined,undetermined,Car,No,Rural,NaN,NaN
806,Name withheld by police,NaN,NaN,Hispanic,NaN,06/04/2019,NM-9,Santa Teresa,NM,88021.0,...,https://www.elpasotimes.com/story/news/immigra...,No,Vehicle,vehicle,NaN,NaN,NaN,Rural,NaN,NaN
807,Name withheld by police,NaN,NaN,Hispanic,NaN,06/04/2019,NM-9,Santa Teresa,NM,88021.0,...,https://www.elpasotimes.com/story/news/immigra...,No,Vehicle,vehicle,NaN,NaN,NaN,Rural,NaN,NaN
1017,Name withheld by police,NaN,NaN,Native American,NaN,23/01/2019,US-491,Naschitti,NM,87325.0,...,https://navajotimes.com/reznews/police-confirm...,No,Unclear,undetermined,undetermined,0,No,Rural,NaN,NaN
1029,Jihad Merrick,NaN,NaN,Unknown race,NaN,16/01/2019,NaN,Nashua,IA,NaN,...,https://wcfcourier.com/news/local/crime-and-co...,Yes,Vehicle,vehicle,attack,Car,No,NaN,Jihad,Merrick


While reading through each article, it would be good to know which columns are Null in case we find that information in the article.  To do this, we'll determine which columns are null for each of the distinct rows, then read the article and impute any information we find.

In [16]:
null_gender_links = null_gender_df['news_article_link']

null_gender_idx = null_gender_links.index
null_gender_urls = null_gender_links.to_list()

print("Here are the links for news articles that contain links for rows with a null gender:")
print("-"*50)
print()
for idx, link in zip(null_gender_idx, null_gender_links):
    null_col_mask = killings.loc[idx].isna()==True
    print(F"df_idx: {idx}")
    print(killings.loc[idx, null_col_mask].to_string())
    print(F"link: {link}")
    print()

Here are the links for news articles that contain links for rows with a null gender:
--------------------------------------------------

df_idx: 13
victims_age       NaN
victims_gender    NaN
victim_img_url    NaN
first_name        NaN
last_name         NaN
link: https://www.jconline.com/story/news/2019/12/30/2-dead-sunday-night-shootings-family-express-near-lafayette/2773121001/

df_idx: 112
victims_age       NaN
victims_gender    NaN
victim_img_url    NaN
first_name        NaN
last_name         NaN
link: https://katu.com/news/local/sheriff-suspect-killed-in-deputy-involved-shooting-in-pierce-county-wash

df_idx: 806
victims_age           NaN
victims_gender        NaN
victim_img_url        NaN
threat_level          NaN
fleeing               NaN
video_surveillance    NaN
first_name            NaN
last_name             NaN
link: https://www.elpasotimes.com/story/news/immigration/2019/04/07/border-patrol-chase-ends-crash-2-migrants-killed-new-mexico/3394408002/

df_idx: 807
victims_age  

In [17]:
# news article says the victim was male, no age given
killings.loc[13, 'victims_gender'] = 'male'

In [18]:
# news article says the victim was male in his 40s
killings.loc[112, ['victims_gender', 'victims_age']] = 'male', '40'

In [19]:
# URL mentions male victim, but ad is behind paywall so can't investigate further
killings.loc[1029, 'victims_gender'] = 'male'

In [20]:
killings.loc[774]

victims_name                               Akeen Brown
victims_age                                        NaN
victims_gender                                     NaN
victims_race                                     Black
victim_img_url                                     NaN
date                                        17/04/2019
street_address                                     NaN
city                                             Flint
state                                               MI
zipcode                                            NaN
county                                             NaN
agency_resp_for_death                              NaN
cause_of_death                                 Gunshot
desc_of_circumstances                              NaN
official_disposition_of_death    Pending investigation
criminal_charges                      No known charges
news_article_link                                  NaN
mental_illness                                     NaN
unarmed   

In [21]:
killings[(killings['victims_gender'].isnull()==True) & (killings['news_article_link'].isnull()==True)]

,victims_name,victims_age,victims_gender,victims_race,victim_img_url,date,street_address,city,state,zipcode,...,news_article_link,mental_illness,unarmed,alleged_weapon,threat_level,fleeing,video_surveillance,geo_type,first_name,last_name
528,Dennis Edwards,NaN,NaN,White,NaN,10/07/2019,NaN,Kansas City,KS,NaN,...,NaN,NaN,Allegedly Armed,gun,attack,Not fleeing,No,NaN,Dennis,Edwards
774,Akeen Brown,NaN,NaN,Black,NaN,17/04/2019,NaN,Flint,MI,NaN,...,NaN,NaN,Allegedly Armed,gun,attack,Not fleeing,No,NaN,Akeen,Brown


In [22]:
# Both names sound male, so I'll set the gender accordingly
killings.loc[528, 'victims_gender'] = 'male'
killings.loc[774, 'victims_gender'] = 'male'

In [23]:
num_null_gender = killings['victims_gender'].isnull().sum()
print(F"There are only {num_null_gender} rows with a null gender left, so we'll impute 'Unknown'")

killings['victims_gender'].fillna('Unknown', inplace=True)

There are only 3 rows with a null gender left, so we'll impute 'Unknown'


In [24]:
killings['victims_gender'] = killings['victims_gender'].str.lower()

### Victim's race

In [25]:
killings['victims_race'].unique()

array(['White', 'Unknown race', 'Black', 'Hispanic', 'Pacific Islander',
       'Asian', 'Native American', 'Unknown Race'], dtype=object)

In [26]:
killings['victims_race'] = killings['victims_race'].str.lower()

In [27]:
killings.loc[killings["victims_race"] == 'unknown race', "victims_race"] = 'unknown'

In [28]:
killings.loc[killings["victims_race"]=='asian', "victims_race"] = 'asian/pacific islander'
killings.loc[killings["victims_race"]=='pacific islander', "victims_race"] = 'asian/pacific islander'

### Victim Image URL

For rows that don't have a URL image of the victim, I'm going to impute "None"

In [29]:
killings['victim_img_url'].fillna('None', inplace=True)

### Location Data

#### City

In [30]:
killings.loc[killings['city'].isnull()==True,['city', 'state', 'zipcode']]

,city,state,zipcode
1755,NaN,ID,NaN
3339,NaN,FL,34639.0
3344,NaN,TX,NaN
3475,NaN,CO,NaN
5561,NaN,FL,32259.0
6511,NaN,AZ,85607.0


Since we have a zipcode and state for several of these rows, we can look up the appropriate city

In [31]:
killings.loc[3339, 'city'] = "Land O' Lakes"
killings.loc[5561, 'city'] = "Jacksonville"
killings.loc[6511, 'city'] = "Douglas"

#### County

In [32]:
killings.loc[killings['county'].isnull()==True,['street_address', 'city', 'state', 'zipcode', 'county']]

,street_address,city,state,zipcode,county
493,Bahalia Rd NE,Wesson,MS,39191.0,NaN
528,NaN,Kansas City,KS,NaN,NaN
774,NaN,Flint,MI,NaN,NaN
1250,NaN,Pratt,KS,NaN,NaN
1305,NaN,Quincy,FL,NaN,NaN
1322,NaN,Quinlan,TX,NaN,NaN
1336,NaN,Orem,OR,NaN,NaN
1346,NaN,Milwaukee,WI,NaN,NaN
1356,NaN,Hayti,MO,NaN,NaN
1367,NaN,Greenback,TN,NaN,NaN


Since we have city and state information for these rows, we can look up the appropriate county

In [33]:
killings.loc[493,'county'] = 'Copiah'
killings.loc[528,'county'] = 'Wyandotte'
killings.loc[774,'county'] = 'Genesee'
killings.loc[1250,'county'] = 'Pratt'
killings.loc[1305,'county'] = 'Gadsden'
killings.loc[1322,'county'] = 'Hunt'
killings.loc[1336,'county'] = 'Utah'
killings.loc[1346,'county'] = 'Milwaukee'
killings.loc[1356,'county'] = 'Pemiscot'
killings.loc[1367,'county'] = 'Loudon'
killings.loc[1430,'county'] = 'Pierce'
killings.loc[1607,'county'] = 'Caldwell'
killings.loc[1965,'county'] = 'Maricopa'
killings.loc[1981,'county'] = 'Daviess'
killings.loc[3315,'county'] = 'Lake'

#### Geography Type/Zipcode/Street Address

In [34]:
killings['geo_type'].unique()

array(['Rural', 'Urban', 'Suburban', nan], dtype=object)

We'll start by looking at placed that have a street address listed so we can use Google maps and try to fill in missing data in these columns

In [35]:
killings.loc[(killings['geo_type'].isnull()==True) & (killings['street_address'] != 'None'), ['street_address', 'city', 'state', 'zipcode', 'geo_type']]

,street_address,city,state,zipcode,geo_type
522,1600 East J St,Tacoma,WA,98421.0,NaN
528,NaN,Kansas City,KS,NaN,NaN
595,1100 Commerce St,Dallas,TX,75242.0,NaN
774,NaN,Flint,MI,NaN,NaN
1000,CA-162 & Aguas Frias Rd,Richvale,CA,95974.0,NaN
...,...,...,...,...,...
6848,1014 N St. SE,Washington,DC,20374.0,NaN
6862,4501 Torley St,Pittsburg,PA,NaN,NaN
6933,Boulder Highway and College Drive,Henderson,NV,89105.0,NaN
7099,NaN,Miami,FL,NaN,NaN


Time to update the null values in the geo_type column.  Note: The data in the data set uses households/sq mi based on zip codes to establish urban, suburban, or rural classification, but for the null values I used households/sq mi based on city instead as the information was more readily available.

- urban: households per square mile >=2213.2 
- suburban: households per square mile >=101.6 and < 2213.2
- rural: households per square mile <101.6

[Source for geo_type classifications](http://jedkolko.com/wp-content/uploads/2015/05/Data-and-methodological-details-052715.pdf)

In [36]:
killings.loc[522, 'geo_type'] = 'Suburban' #82540 households / 62.42 sq mi = 1322 households / sq mi (suburban)

In [37]:
killings.loc[595, 'geo_type'] = 'Suburban' #521198 households / 385.8 sq mi = 1351 households / sq mi (suburban)

In [38]:
killings.loc[1000, 'geo_type'] = 'Rural'

In [39]:
killings.loc[1004, 'geo_type'] = 'Rural'

In [40]:
killings.loc[1281, 'geo_type'] = 'Suburban' # 156,482 households / 156.6 sq. mi = 999 households / sq mi (suburban)

In [41]:
killings.loc[1947, 'geo_type'] = 'Suburban' # 48095 households / 108.3 sq. mi = 444 households / sq mi (suburban)

In [42]:
killings.loc[2072, 'geo_type'] = 'Suburban' # 33 households / .22 sq. mi = 150 households / sq mi (suburban, but just barely)

In [43]:
killings.loc[2207, 'geo_type'] = 'Suburban' # 321835 households / 181.4 sq. mi = 1774 households / sq mi (suburban, almost urban)

In [44]:
killings.loc[2419, 'geo_type'] = 'Urban' # 130885 households / 22.78 sq. mi = 5745 households / sq mi (urban)

In [45]:
killings.loc[2488, 'geo_type'] = 'Suburban' # 23 households / .14 sq. mi = 164 households / sq mi (Suburban, just barely)

In [46]:
killings.loc[3315, 'geo_type'] = 'Suburban' # 7013 households / 5.598 sq. mi = 1263 households / sq mi (suburban)

In [47]:
killings.loc[3347, 'geo_type'] = 'Suburban' # 3203 households / 7.14 sq. mi = 449 households / sq mi (Suburban)

In [48]:
killings.loc[3581, 'geo_type'] = 'Suburban' # 125894 households / 740 sq. mi = 170 households / sq mi (Suburban)

The next location is at Foxwoods Casino in CT, which is technically part of Mashantucket CT., I think because it's considered an indian reservation.  The casino is surrounded by Ledyard CT. and it might be a better idea to use that city as it may be more representative of the household popluation size and square mileage need to classify the geo_type.

In [49]:
killings.loc[3621, 'geo_type'] = 'Rural' # 62 households / 2.6 sq. mi = 24 households / sq mi (Rural)

In [50]:
killings.loc[3699, 'geo_type'] = 'Rural' # 115 households / 1.5 sq. mi = 77 households / sq mi (Rural)

In [51]:
killings.loc[3740, 'geo_type'] = 'Suburban' # 521198 households / 385.8 sq. mi = 1351 households / sq mi (Suburban)

In [52]:
killings.loc[4409, ['geo_type', 'zipcode']] = 'Suburban', 32218.0 #359607 households / 875 sq mi = 411 (Suburban)

In [53]:
killings.loc[4535, ['geo_type', 'zipcode']] = 'Suburban', 46368.0 #13992 households / 27.61 sq mi = 507 (Suburban)

In [54]:
killings.loc[4571, ['geo_type', 'zipcode']] = 'Suburban', 97210.0 #264428 households / 145 sq mi = 1824 (Suburban)

In [55]:
killings.loc[[4592, 4593], ['geo_type', 'zipcode']] = 'Suburban', 77014.0 #848340 households / 669 sq mi = 1268 (Suburban)

In [56]:
killings.loc[4594, ['geo_type', 'zipcode']] = 'Suburban', 74434.0 #1639 households / 14.13 sq mi = 116 (Suburban)

In [57]:
killings.loc[4640, ['geo_type', 'zipcode']] = 'Suburban', 30680.0 #5337 households / 14.15 sq mi = 377 (Suburban)

In [58]:
killings.loc[5021, 'geo_type'] = 'Rural' # 268 households / 8.842 sq. mi = 30 households / sq mi (Rural)

In [59]:
killings.loc[5164, ['geo_type', 'street_address']] = 'Rural', '182 N 4430 Rd' # No census data on household population, but on Google maps it looks very rural

In [60]:
killings.loc[5192, 'geo_type'] = 'Suburban' # 39122 households / 22.99 sq. mi = 1702 households / sq mi (Suburban)

In [61]:
killings.loc[5268, 'geo_type'] = 'Suburban' # 355 households / 0.74 sq. mi = 480 households / sq mi (Suburban)

In [62]:
killings.loc[5371, ['geo_type', 'zipcode']] = 'Suburban', 77073.0 #848340 households / 669 sq mi = 1268 (Suburban)

In [63]:
killings.loc[5623, 'geo_type'] = 'Suburban' # 63217 households / 103.1 sq. mi = 613 households / sq mi (Suburban)

In [64]:
killings.loc[5709, 'geo_type'] = 'Rural' # Jean is just outside of Las Vegas.  Has no residents but is considered a commercial town.  Seems rural enough.

In [65]:
killings.loc[5805, 'geo_type'] = 'Suburban' # 3061 households / 13.13 sq. mi = 233 households / sq mi (Suburban)

In [66]:
killings.loc[[4451, 6080], 'geo_type'] = 'Urban' # 323446 households / 142.5 sq. mi = 2270 households / sq mi (Urban)

In [67]:
killings.loc[6188, 'geo_type'] = 'Urban' # 7229 households / 0.648 sq. mi = 11156 households / sq mi (Urban)\

In [68]:
killings.loc[6570, ['street_address', 'zipcode', 'geo_type']] = '12097 Veterans Memorial Dr', 77067.0, 'Suburban' #848340 households / 669 sq mi = 1268 (Suburban)

In [69]:
killings.loc[6442, 'geo_type'] = 'Rural' # Outskirts of Las Vegas, seems very Rural

In [70]:
killings.loc[6573, ['geo_type', 'zipcode', 'city']] = 'Suburban', 73104.0, 'Oklahoma City' # 240471 / 621 = 387 (Suburban)

In [71]:
killings.loc[6637, ['geo_type', 'zipcode']] = 'Suburban', 70767.0 # 2162 / 3.328 = 650 (Suburban)

In [72]:
killings.loc[6643, ['street_address', 'zipcode', 'geo_type']] = '32000 Westport Way', 92596.0, 'Suburban' # 8539 / 10.9 = 783 (Suburban)

In [73]:
killings.loc[6697, ['street_address', 'geo_type']] = '2335 Union Dr', 'Suburban' # 25243 / 224.27 = 113 (Suburban)

In [74]:
killings.loc[6746, 'geo_type'] = 'Suburban' # 199478 / 136.8 = 1458 (Suburban)

In [75]:
killings.loc[6848, 'geo_type'] = 'Urban' # 281322 / 68.34 = 4117 (Urban)

In [76]:
killings.loc[6862, ['zipcode', 'geo_type']] = 15224.0, 'Urban' # 136275 / 58.34 = 2336

In [77]:
killings.loc[6933, 'geo_type'] = 'Suburban' # 113901 / 108 = 1055 (Suburban)

In [78]:
killings.loc[killings['geo_type'].isnull()==True, ['street_address', 'city', 'state', 'zipcode', 'geo_type', 'news_article_link']]

,street_address,city,state,zipcode,geo_type,news_article_link
528,NaN,Kansas City,KS,NaN,NaN,NaN
774,NaN,Flint,MI,NaN,NaN,NaN
1029,NaN,Nashua,IA,NaN,NaN,https://wcfcourier.com/news/local/crime-and-co...
1250,NaN,Pratt,KS,NaN,NaN,https://www.kansas.com/news/local/crime/articl...
1305,NaN,Quincy,FL,NaN,NaN,NaN
1322,NaN,Quinlan,TX,NaN,NaN,NaN
1336,NaN,Orem,OR,NaN,NaN,NaN
1346,NaN,Milwaukee,WI,NaN,NaN,NaN
1356,NaN,Hayti,MO,NaN,NaN,NaN
1367,NaN,Greenback,TN,NaN,NaN,NaN


In [79]:
killings.loc[528, 'geo_type'] = 'Suburban' # 53925 / 128.4 = 420 (Suburban)

In [80]:
killings.loc[774, 'geo_type'] = 'Suburban' # 40035 / 34.11 = 1174 (Suburban)

In [81]:
killings.loc[1029, 'geo_type'] = 'Suburban' # 682 / 3.14 = 217 (Suburban)

In [82]:
killings.loc[1250, ['street_address', 'zipcode', 'geo_type']] = '500 N Main St', 67124.0, 'Suburban' # 2837 / 7.49 = 379 (Suburban)

In [83]:
killings.loc[1305, 'geo_type'] = 'Suburban' # 2810 / 11.54 = 244 (Suburban)

In [84]:
killings.loc[1322, 'geo_type'] = 'Suburban' # 574 / 1.324 = 434 (Suburban)

In [85]:
killings.loc[1336, 'geo_type'] = 'Suburban' # 28177 / 18.57 = 1517 (Suburban)

In [86]:
killings.loc[1346, 'geo_type'] = 'Urban' # 229556 / 96.81 = 2371 (Urban)

In [87]:
killings.loc[1356, 'geo_type'] = 'Suburban' # 1258 / 2.31 = 545 (Suburban)

In [88]:
killings.loc[1367, 'geo_type'] = 'Rural' # 394 / 8.452 = 47 (Rural)

In [89]:
killings.loc[1430, 'geo_type'] = 'Suburban' # 10780 / 8.687 = 1241 (Suburban)

In [90]:
killings.loc[1607, 'geo_type'] = 'Suburban' # 23121 / 19.7 = 1174 (Suburban)

In [91]:
killings.loc[1812, ['geo_type']] = 'Rural' # 32 / .4 = 80 (Rural)

In [92]:
killings.loc[1965, 'geo_type'] = 'Suburban' # 111221 / 184.4 = 603 (Suburban)

In [93]:
killings.loc[1981, 'geo_type'] = 'Suburban' # 118 / 1.05 = 112 (Suburban)

In [94]:
killings.loc[2813, ['street_address', 'zipcode', 'geo_type']] = '6800 62nd Ave NE', 98115.0, 'Urban' # 283510 / 83.78 = 3384 (Urban)

In [95]:
killings.loc[3344, ['city', 'zipcode', 'geo_type']] = 'Campbellton', 78008.0, 'Rural' # 176 / 198.5 = 1 (Rural)

In [96]:
killings.loc[3346, ['zipcode', 'geo_type']] = 57752.0, 'Rural' # 177 / 2.008 = 88 (Rural)

In [97]:
killings.loc[3475, ['street_address', 'zipcode', 'geo_type', 'city']] = 'X4 Rd', 81411.0, 'Rural', 'Bedrock' #Out in the middle of nowhere

In [98]:
killings.loc[6812, 'geo_type'] = 'Suburban' # 4034 / 17.1 = 236 (Suburban)

In [99]:
killings.loc[7099, 'geo_type'] = 'Urban' # 870051 / 55.25 = 15748 (Urban)

In [100]:
killings.loc[7461, 'geo_type'] = 'Suburban' # 8689 / 4.36 = 1993 (Suburban)

#### Handling location fields that are still Null
Since we've done as much as we can in the way of researching location data, we'll replace all null values that remain with 'unknown'

In [101]:
num_null_street_address = killings['street_address'].isnull().sum()
print(F"There are {num_null_street_address} null street addresses that will be filled with 'unknown'")
killings['street_address'].fillna('unknown', inplace=True)

There are 80 null street addresses that will be filled with 'unknown'


In [102]:
killings.loc[killings['street_address']=='Unknown', 'street_address'] = 'unknown'

In [103]:
num_null_city = killings['city'].isnull().sum()
print(F"There are {num_null_city} cities that will be filled with 'unknown'")

killings['city'].fillna('unknown', inplace=True)

There are 1 cities that will be filled with 'unknown'


In [104]:
# num_null_zip_codes = killings['zipcode'].isnull().sum()
# print(F"There are {num_null_zip_codes} zip_codes that will be filled with 'unknown'")

# killings['zipcode'].fillna('unknown', inplace=True)

In [105]:
num_null_geo_type = killings['geo_type'].isnull().sum()
print(F"There are {num_null_geo_type} geo_types that will be filled with 'unknown'")

killings['geo_type'].fillna('unknown', inplace=True)

There are 1 geo_types that will be filled with 'unknown'


### Agency Responsible for death

In [106]:
num_null_agency_responsible = killings['agency_resp_for_death'].isnull().sum()
print(F"There are {num_null_agency_responsible} rows for agency responsible for death that will be filled with 'unknown'")

killings['agency_resp_for_death'].fillna('unknown', inplace=True)

There are 16 rows for agency responsible for death that will be filled with 'unknown'


### Cause of death
There are too many categories here to do a meaningful analysis, I'll distill them down to just a few categories

In [107]:
killings['cause_of_death'].unique()

array(['Gunshot', 'Gunshot, Taser', 'Physical Restraint', 'Taser',
       'Beaten/Bludgeoned with instrument', 'Beaten', 'Vehicle',
       'Asphyxiated', 'Other', 'Gunshot, Beanbag Gun',
       'Gunshot, Police Dog', 'Gunshot, Bean Bag Gun',
       'Gunshot, Pepper Spray', 'Bean bag', 'Gunshot, Taser, Baton',
       'Gunshot, Taser, Beanbag Shotgun', 'Pepper Spray',
       'Gunshot, Unspecified Less Lethal Weapon', 'Physical restraint',
       'Taser, Baton', 'Bomb', 'Taser, Physical Restraint',
       'Baton, Pepper Spray, Physical Restraint', 'Gunshot, Vehicle',
       'Gunshot, Stabbed', 'Gunshot, Taser, Pepper spray',
       'Taser, Pepper Spray, Beaten', 'Taser, Beaten',
       'Taser, Pepper spray, beaten', 'Tasered'], dtype=object)

In [108]:
killings['cause_of_death'] = killings['cause_of_death'].str.lower()

In [109]:
cause_of_death_dict = {"gunshot, bean bag gun":"gunshot, beanbag gun",
                       "tasered":"taser",
                       "beaten/bludgeoned with instrument":"beaten",
                       "gunshot, taser":"gunshot",
                       "gunshot, police dog":"gunshot",
                       "gunshot, pepper spray":"gunshot",
                       "gunshot, beanbag gun":"gunshot",
                       "taser, pepper spray, beaten":"taser",
                       "taser, physical restraint":"taser",
                       "gunshot, taser, pepper spray":"gunshot",
                       "gunshot, stabbed":"gunshot",
                       "gunshot, vehicle":"gunshot",
                       "gunshot, taser, baton":"gunshot",
                       "gunshot, unspecified less lethal weapon":"gunshot",
                       "gunshot, taser, beanbag shotgun":"gunshot",
                       "gunshot, taser, beanbag shotgun":"gunshot",
                       "taser, baton":"taser",
                       "taser, beaten":"taser",
                       "bomb":"other",
                       "baton, pepper spray, physical restraint":"other",
                       "pepper spray":"other",
                       "bean bag":"other"}

killings['cause_of_death'] = killings['cause_of_death'].map(cause_of_death_dict).fillna(killings['cause_of_death'])

In [110]:
killings['cause_of_death'].value_counts()

gunshot                 7298
taser                    253
vehicle                   33
beaten                    31
physical restraint        20
asphyxiated               14
other                     13
gunshot, beanbag gun       1
Name: cause_of_death, dtype: int64

### Description of Circumstances Surrounding Death
It'll take too much time to research these missing entries and I don't think they'll contribute much to analysis, so I'll fill them with "Unavailable"

In [111]:
killings['desc_of_circumstances'].fillna('Unavailable', inplace=True)

### Official Disposition of Death

In [112]:
killings['official_disposition_of_death'].unique()

array(['Pending investigation', 'Justified by Prosecuting Attorney',
       'Criminal', 'Justified by County Prosecutor',
       'Justified by District Attorney', 'Unreported',
       "Justified by State Attorney's Office",
       "Justified by State's Attorney", 'Justified by County Attorney',
       'Justified by outside agency', 'Charged with a crime',
       'Administrative discipline', 'Murder/suicide',
       'Justified by internal review', 'Charged with manslaughter',
       'Justified by Prosecutor', 'Officer indicted/Pending', nan,
       'Charged with murder', 'Charged with voluntary manslaughter',
       'Charged', 'Charged, Convicted of Third Degree Murder',
       'Charged with involuntary manslaughter',
       'Charged with first-degree murder', 'Charged, Convicted',
       'Accidental', 'Charged with murder, Acquitted',
       'Charged with second-degree murder',
       'Charged with reckless homicide', 'Charged, charges dismissed',
       'Justified', 'Charged with mans

There are way too many unique entries in this column, let's reduce the number of categories down

In [113]:
# off_disp_dict = {"pending investigaton":"pending investigation",
#                  "ongoing investigation":"under investigation",
#                  }

In [114]:
num_null_disp_of_death = killings['official_disposition_of_death'].isnull().sum()
print(F"There are {num_null_disp_of_death} rows for official disposition of death that will be filled with 'unknown'")

killings['official_disposition_of_death'].fillna('unknown', inplace=True)

There are 256 rows for official disposition of death that will be filled with 'unknown'


In [115]:
killings.loc[killings['official_disposition_of_death'].str.lower().str.contains('unjustified'), 'official_disposition_of_death'] = 'unjustified'

In [116]:
killings.loc[killings['official_disposition_of_death'].str.lower().str.contains('justified'), 'official_disposition_of_death'] = 'justified'

In [117]:
killings.loc[killings['official_disposition_of_death'].str.lower().str.contains('convicted'), 'official_disposition_of_death'] = 'convicted'

In [118]:
killings.loc[killings['official_disposition_of_death'].str.lower().str.contains('acquitted'), 'official_disposition_of_death'] = 'acquitted'

In [119]:
killings.loc[killings['official_disposition_of_death'].str.lower().str.contains('pending investigation'), 'official_disposition_of_death'] = 'pending investigation'

In [120]:
killings.loc[killings['official_disposition_of_death'].str.lower().str.contains('pending investigaton'), 'official_disposition_of_death'] = 'pending investigation'

In [121]:
killings.loc[killings['official_disposition_of_death'].str.lower().str.contains('ongoing investigation'), 'official_disposition_of_death'] = 'under investigation'

In [122]:
killings.loc[killings['official_disposition_of_death'].str.lower().str.contains('under investigation'), 'official_disposition_of_death'] = 'under investigation'

In [123]:
killings.loc[killings['official_disposition_of_death'].str.lower().str.contains('no indictment'), 'official_disposition_of_death'] = 'no indictment'

In [124]:
killings.loc[killings['official_disposition_of_death'].str.lower().str.contains('indicted'), 'official_disposition_of_death'] = 'indicted'

In [125]:
killings.loc[killings['official_disposition_of_death'].str.lower().str.contains('charged'), 'official_disposition_of_death'] = 'charged'

In [126]:
killings.loc[killings['official_disposition_of_death'].str.lower().str.contains('No charges'), 'official_disposition_of_death'] = 'no charges'

In [127]:
killings.loc[killings['official_disposition_of_death'].str.lower().str.contains('no charges'), 'official_disposition_of_death'] = 'no charges'

In [128]:
killings.loc[killings['official_disposition_of_death'].str.lower().str.contains('no known charges'), 'official_disposition_of_death'] = 'no charges'

In [129]:
killings.loc[killings['official_disposition_of_death'].str.lower().str.contains('unreported'), 'official_disposition_of_death'] = 'unreported'

In [130]:
killings.loc[killings['official_disposition_of_death'].str.lower().str.contains('unknown'), 'official_disposition_of_death'] = 'unknown'

In [131]:
killings.loc[killings['official_disposition_of_death'].str.lower().str.contains('unknown'), 'official_disposition_of_death'] = 'unknown'

In [132]:
killings['official_disposition_of_death'] = killings['official_disposition_of_death'].str.lower()

In [133]:
killings['official_disposition_of_death'].value_counts()

pending investigation            5136
unreported                       1140
justified                         687
unknown                           510
charged                            52
murder/suicide                     25
convicted                          25
criminal                           22
under investigation                17
grand jury/no bill or cleared      14
acquitted                          12
found excusable                     6
no charges                          5
indicted                            3
found not guilty                    2
cleared by district attorney        2
no indictment                       2
insufficient evidence               1
administrative discipline           1
accidental                          1
Name: official_disposition_of_death, dtype: int64

### Criminal Charges

In [134]:
killings['criminal_charges'].value_counts()

No known charges                                                                 7524
Charged with a crime                                                               49
No                                                                                 37
Charged, Acquitted                                                                 17
Charged, Mistrial                                                                   5
Charged, Convicted                                                                  4
Charged, Convicted, Sentenced to 5 years probation.                                 2
Charged, Charges Tossed                                                             2
Charged, Convicted, Sentenced to life in prison                                     2
Charged, Convicted, Sentenced to 30 years in prison                                 2
Charged, Convicted, Sentenced to 1 year in prison                                   1
Charged, Convicted, Sentenced to 3 years probation    

In [135]:
killings['criminal_charges'] = killings['criminal_charges'].str.lower()

In [136]:
killings.loc[killings['criminal_charges'] == 'no known charges', 'criminal_charges'] = 'no charges'
killings.loc[killings['criminal_charges'] == 'no', 'criminal_charges'] = 'no charges'

In [137]:
killings.loc[killings['criminal_charges'].str.contains('charged, convicted'), 'criminal_charges'] = 'charged, convicted'

In [138]:
killings.loc[killings['criminal_charges'].str.contains('charged, mistrial'), 'criminal_charges'] = 'charged, mistrial'

In [139]:
killings.loc[killings['criminal_charges'].str.contains('charged, charges tossed'), 'criminal_charges'] = 'charged, charges dropped'

In [140]:
killings.loc[killings['criminal_charges'].str.contains('charged with manslaughter'), 'criminal_charges'] = 'charged with a crime'

In [141]:
killings['criminal_charges'].value_counts()

no charges                  7562
charged with a crime          50
charged, convicted            25
charged, acquitted            17
charged, mistrial              6
charged, charges dropped       3
Name: criminal_charges, dtype: int64

### News Article Link
It's going to be near impossible to research links for these rows, so we'll also fill them with "Unavailable"

In [142]:
killings['news_article_link'].fillna('Unavailable', inplace=True)

### Symptoms of Mental Illness

In [143]:
killings['mental_illness'].unique()

array(['No', 'Unknown', 'Yes', 'Drug or alcohol use', nan, 'Unkown',
       'Unknown ', 'unknown'], dtype=object)

Because "Unknown", "Unkown", "Unknown ", and "unknown" all represent the same idea, we need to distill them down to a single category.

In [144]:
killings['mental_illness'] = killings['mental_illness'].str.lower()

In [145]:
mental_illness_dict = {"unkown":"unknown",
                       "unknown ":"unknown"}

killings['mental_illness'] = killings['mental_illness'].map(mental_illness_dict).fillna(killings['mental_illness'])

In [146]:
num_null_killings = killings['mental_illness'].isnull().sum()
print(F"There are {num_null_killings} null rows for mental_illness that will be filled with 'unknown'")

killings['mental_illness'].fillna('unknown', inplace=True)

There are 11 null rows for mental_illness that will be filled with 'unknown'


### Unarmed

In [147]:
killings['unarmed'].unique()

array(['Allegedly Armed', 'Unclear', 'Vehicle', 'Unarmed'], dtype=object)

In [148]:
killings['unarmed'] = killings['unarmed'].str.lower()

### Alleged Weapon - need to do

In [163]:
killings['alleged_weapon'].unique()

array(['knife', 'unknown', 'gun', 'vehicle', 'chain', 'taser',
       'gun and vehicle', 'shovel', 'gun and car', 'toy weapon',
       'unarmed', 'baseball bat', 'hatchet', 'car, knife and mace',
       'sword', 'vehicle and machete', 'screwdriver', 'vehicle and gun',
       'bb gun', 'sharp object', 'box cutter', 'ax', 'hammer', 'crowbar',
       'chair', 'scissors', 'toy', 'baseball bat and knife',
       'straight edge razor', 'machete', 'baton', 'airsoft pistol',
       'air pistol', 'wasp spray', 'bb gun and vehicle', 'piece of wood',
       'gun and knife', 'garden tool', 'barstool', 'walking stick',
       'wrench', 'beer bottle', 'meat cleaver', 'metal pipe', 'flag pole',
       'rock', 'lawn mower blade', 'crossbow', 'metal object',
       'bow and arrow', 'pick-axe', 'lamp', 'glass shard',
       'incendiary device', 'pipe', 'pole and knife', 'lighter fluid',
       'pitchfork', 'gun and sword', 'blunt object', 'chainsaw', 'pen',
       'fireworks', 'gun and hatchet', 'baseba

In [160]:
killings['alleged_weapon'] = killings['alleged_weapon'].str.lower()

In [162]:
alleged_weapon_dict = {"unclear":"unknown",
                       "unknown weapon":"unknown",
                       "undetermined":"unknown",
                       }

killings['alleged_weapon'] = killings['alleged_weapon'].map(alleged_weapon_dict).fillna(killings['alleged_weapon'])

### Threat Level

In [149]:
killings['threat_level'].unique()

array(['other', 'attack', nan, 'undetermined'], dtype=object)

In [150]:
num_null_threat_level = killings['threat_level'].isnull().sum()
print(F"There are {num_null_threat_level} rows for threat level that will be filled with 'unknown'")

killings['threat_level'].fillna('unknown', inplace=True)

There are 2382 rows for threat level that will be filled with 'unknown'


### Fleeing

In [153]:
killings['fleeing'].unique()

array(['not fleeing', 'unknown', 'car', 'foot', 'other'], dtype=object)

In [152]:
killings['fleeing'] = killings['fleeing'].str.lower()

killings.loc[killings['fleeing']=='0', 'fleeing'] = 'unknown'
# fleeing_dict = {'Foot':'foot',
#                 'Not fleeing':'not fleeing',
#                 '0':'unknown',
#                 'Car':'car',
#                 'Other':'other'}

# killings['fleeing'] = killings['fleeing'].map(fleeing_dict).fillna(killings['fleeing'])
killings['fleeing'].fillna('unknown', inplace=True)

### Video surveillance

In [156]:
killings['video_surveillance'].unique()

array(['no', 'unknown', 'body camera', 'bystander video',
       'surveillance video'], dtype=object)

In [155]:
killings['video_surveillance'] = killings['video_surveillance'].str.lower()
killings['video_surveillance'] = killings['video_surveillance'].str.replace('yes', 'body camera')
killings['video_surveillance'].fillna('unknown', inplace=True)

## Converting data types

In [ ]:
killings.dtypes

### Age

In [ ]:
killings["victims_age"] = killings["victims_age"].astype('float64')

### Date

We can convert the dates in the date column to Python datetime objects

In [ ]:
killings["date"] = pd.to_datetime(killings["date"], infer_datetime_format=True)

In [ ]:
killings.dtypes

In [ ]:
killings.columns

In [ ]:
killings['unarmed'].unique()

In [ ]:
killings['alleged_weapon'].unique()

In [ ]:
zipcode_null = killings.loc[killings['Zipcode'].isnull()==True,:].index
killings.drop(zipcode_null,inplace=True)

In [ ]:
killings.isnull().sum()

In [ ]:
killings.head()

In [ ]:
killings.head()

In [ ]:
killings.to_csv('./csv_files/police_killings_clean.csv', index=False)

# Scratch Work

In [ ]:
# This code will open all links so we can inspect them manually
try:
    for url in null_gender_urls:
        webbrowser.open_new_tab(url)
except:
    pass

In [ ]:
import urllib.request

def is_url_working(x):
    import urllib.request
    print('starting')
    try:
        test_url = urllib.request.urlopen(x)
        return test_url
    except:
        return 'No'
    
# killings['URL working?'] = killings['Link to news article or photo of official document'].apply(is_url_working)

In [ ]:
null_address_df = killings[(killings["street_address"].isnull() == True) & (killings["news_article_link"].isnull()==False)]

null_address_links = null_address_df['news_article_link']

null_address_idx = null_address_links.index
null_address_urls = null_address_links.to_list()

print("Here are the links for news articles that contain links for rows with a null street_address:")
print("-"*50)
print()
for idx, link in zip(null_address_idx[0:10], null_address_links[0:10]):
    null_col_mask = killings.loc[idx].isna()==True
    print(F"df_idx: {idx}")
    print(killings.loc[idx, null_col_mask].to_string())
    print(F"link: {link}")
    print()